**Import packages**

In [1]:
import warnings
# Import TF and TF Hub libraries.
import tensorflow as tf
import tensorflow_hub as hub
#GUI
import PySimpleGUI as sg
import os, csv, tqdm, cv2, time
from PIL import Image
import numpy as np
import pandas as pd
import shutil
# Import Classifiers
from sklearn.model_selection import train_test_split as tts
from sklearn import svm
from sklearn import metrics

In [2]:
warnings.filterwarnings("ignore")

**Path to directories**

In [3]:
directory = 'C:\\Users\\rache\\Documents\\'

In [4]:
# Path to the folder containing the input images
images_folder = directory +'Rehabilitation_interface\\BodyEvaluation\\Data\\' #1
images_out_folder = directory +'Rehabilitation_interface\\BodyEvaluation\\DataOutputMovenet\\' #2
## Obtener rutas de las imágenes
resized_path = directory +'Rehabilitation_interface\\BodyEvaluation\\resized\\' #3
# Path to the output CSV file
csv_out_path = directory +'Rehabilitation_interface\\BodyEvaluation\\DataOutputMovenet\\fitness_poses_csvs_out_basic_movenet.csv' #4
# Download the model from TF Hub.
model_path = directory +'Rehabilitation_interface' #5
#Initializing the model
model = tf.saved_model.load(model_path)
movenet = model.signatures['serving_default']
path_to_data = directory +'Rehabilitation_interface\\BodyEvaluation\\Data'
path_to_classes = directory +'Rehabilitation_interface\\BodyEvaluation\\Data\\clase'

**Functions of the interface**

First GUI

In [5]:
#Layout definition
def create_classes_layout():
    ## Define the layout for the "Create Classes" window
    layout = [
        [sg.Text('Number of classes:'), sg.InputText(default_text='2',size=(10,1))],
        [sg.Text('Number of photos per class:'), sg.InputText(default_text='5',size=(10,1))],
        [sg.Button('Create Classes')],
        [sg.Text(size=(40,1), key='-OUTPUT-')],
        [sg.Image(filename='', key='-IMAGE-', size=(30,15))],
        [sg.Button('Close window')],
    ]
    return layout

#Creation of windows
def create_classes_window():
    ## Create the "Create Classes" window
    layout = create_classes_layout()
    window = sg.Window('Pose Classifier - Create Classes', layout)
    return window

#Creation of the classes with the interface
def create_classes(window, path_to_data, path_to_classes, num_classes, num_photos, time_between_photos):
    # Remove all directories that start with "clase"
    for root, dirs, files in os.walk(path_to_data):
        for dir in dirs:
            if dir.startswith("clase"):
                shutil.rmtree(os.path.join(root, dir))
    for clase in range(num_classes):
        # Wait for the user to be ready
        sg.popup('Haz la posición número {}. Pulsa OK cuando estés listo.'.format(clase+1), title='¡Listo!')
        window['-OUTPUT-'].update('Haz la posición número {}. Esperando...'.format(clase+1))
        window.refresh()
        time.sleep(3)
        folder_name = str(path_to_classes+str(clase))
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        # Loop to take the photos
        for i in range(num_photos):
            window['-OUTPUT-'].update('Foto {} de {}'.format(i+1,num_photos))
            window.refresh()
            # Read a frame from the webcam
            ret, frame = camera.read()
            # Convert the image to PNG Bytes
            resized = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            imgbytes = cv2.imencode('.png', resized)[1].tobytes()
            # Check if the camera is working
            if not ret:
                window['-OUTPUT-'].update('La cámara no funciona!')
                window.refresh()
                break
            # Update the image element in the GUI with the new image
            window['-IMAGE-'].update(data=imgbytes)
            # Generate the file name for the current picture
            filename = folder_name+'/photo_{}.png'.format(i)
            # Save the frame to a file
            cv2.imwrite(filename, frame)
            # Wait for the specified time before taking the next photo
            time.sleep(time_between_photos)

Second GUI

In [6]:
# Change size images to see them on the interface 
def class_image_resized():
    image_paths = []

    for folder_name in os.listdir(images_folder):
        if folder_name.startswith("clase"):
            image_paths.append(os.path.join(images_folder, folder_name, "photo_0.png"))
    for i, image_path in enumerate(image_paths):
        img = Image.open(image_path)
        img_resized = img.resize((100, 100))
        resized_image_path = os.path.join(resized_path, f"clase{i}.png")
        img_resized.save(resized_image_path)
    
    image_columns = []
    for i in range(len(image_paths)):
        image_column = sg.Column([[sg.Image(filename = os.path.join(resized_path,f"clase{i}.png"))]])#, pad = (0,0))
        image_columns.append(image_column)
    return image_columns

#Buttons to train and classify the models
def create_training_layout():
    image_columns = class_image_resized()
    # Creamos un layout con una fila y las columnas de las imágenes
    layout = [image_columns]
    # Add remaining GUI elements
    layout += [
        [sg.Text('The rehabiliation is fixed on Full body')],
        #[sg.Checkbox('Full body:', default=True, key="-Full body-"),sg.Checkbox('Cara:', default=False, key="-Cara-"),sg.Checkbox('Torso:', default=False, key="-Torso-"),sg.Checkbox('Piernas:', default=False, key="-Piernas-"),sg.Checkbox('Puntos fiables:', default=False, key="-Puntos fiables-")],
        [sg.Button('Train Model')],
        [sg.Text(size=(50,1), key='-1-')],
        [sg.Button('Classify Pose from Camera')],
        [sg.Text(size=(50,1), key='-2-')],
        #[sg.Button('Try Real Time'), sg.Button('STOP', button_color=('white', 'red'))],
        #[sg.Text(size=(50,1), key='-3-')],
        [sg.Button('Close Window')]
    ]
    return layout

#Corret format of the indexes
def keypoints_format(list_points):
    #list_points = [list_points[i]-1 for i in range(len(list_points))]
    idx = pd.Index(list(map(str, list_points)))
    return idx
def keypoints_in_range(list_points):
    new_idx = [int(list_points[i])-1 for i in range(len(list_points))]
    updated_idx = pd.Index(new_idx)
    return updated_idx
#Choose keypoints to detect
def keypoints_per_part_of_body(values):
    df = pd.read_csv(csv_out_path)    
    list_points = [1, 2, 3, 4, 5, 6,7,8,9,10,11,12,13,14,15,16,17]
    idx = keypoints_format(list_points)    
    numbers = df.columns.str.extract(r'(x|y)(\d+)')[1] #obtenemos los valores numericos de las columnas
    cols_to_keep = df[df.columns[df.columns.isin(['name', 'class']) | numbers.isin(idx)]]
    cols_to_keep.to_csv(csv_out_path, index=False) #guardamos el dataframe final en el fichero csv_out_path sobrescribiéndolo
    df = pd.read_csv(csv_out_path)
    #display(df)
    idx = keypoints_in_range(idx)
    return idx

#Second windows
def create_training_window():
    ## Create the "Create Classes" window
    layout = create_training_layout()
    window = sg.Window('Ejemplo de imágenes', layout)
    return window

# Define the function to extract landmarks from an image
def extract_landmarks(image_path):
    #Load images
    image = tf.io.read_file(image_path)
    image = tf.compat.v1.image.decode_jpeg(image)
    image = tf.expand_dims(image, axis=0)
    # Resize and pad the image to keep the aspect ratio and fit the expected size.
    image = tf.cast(tf.image.resize_with_pad(image, 192, 192), dtype=tf.int32)

    # Run model inference.
    outputs = movenet(image)
    # Output is a [1, 1, 17, 3] tensor.
    #keypoints = outputs['output_0'] # 17 keypoints
    #only 10 points
    #keypoints = outputs['output_0'][:, :, :10, :]  # Only take the first 10 keypoints
    keypoints = outputs['output_0']

    return keypoints

# Extract features from images
def extract_features_to_train(csv_out_file, images_folder, images_out_foulder):
    csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    # Folder names are used as pose class names.
    pose_class_names = sorted([n for n in os.listdir(images_folder) if not n.startswith('.')])
    # Write the header row to the CSV file
    csv_out_writer.writerow(['name', 'class', 'x1', 'y1', 'p1', 'x2', 'y2', 'p2', 'x3', 'y3', 'p3', 'x4', 'y4', 'p4', 'x5', 'y5', 'p5', 'x6', 'y6', 'p6', 'x7', 'y7', 'p7', 'x8', 'y8', 'p8', 'x9', 'y9', 'p9', 'x10', 'y10', 'p10', 'x11', 'y11', 'p11', 'x12', 'y12', 'p12', 'x13', 'y13', 'p13', 'x14', 'y14', 'p14', 'x15', 'y15', 'p15', 'x16', 'y16', 'p16', 'x17', 'y17', 'p17'])  # Header row
    for pose_class_name in pose_class_names:
        message = f'Bootstrapping {pose_class_name}'
        window['-1-'].update(message)
        if not os.path.exists(os.path.join(images_out_folder, pose_class_name)):
            os.makedirs(os.path.join(images_out_folder, pose_class_name))
        image_names = sorted([
        n for n in os.listdir(os.path.join(images_folder, pose_class_name))
        if not n.startswith('.')])
        new_row = []
        for image_name in tqdm.tqdm(image_names, position=0):
            new_row = []
            # Extract the landmarks from the image
            landmarks = extract_landmarks(os.path.join(images_folder,pose_class_name, image_name))
            # Assuming you have the pose data in a variable named `pose`
            pose_np = np.squeeze(landmarks.numpy())  # Remove the batch and channel dimensions
            pose_np = pose_np.transpose()
            # Write pose sample to CSV.
            new_row.append(image_name)
            new_row.append(pose_class_name)
            for i in range(pose_np.shape[1]):
                x = pose_np[0,i]
                y = pose_np[1,i]
                p = pose_np[2,i]
                new_row.append(x)
                new_row.append(y)
                new_row.append(p)
            # Save the landmarks to the CSV file
            csv_out_writer.writerow(new_row)
    # Aqui quiero cambiar todo el fichero csv_out_path y sobreescribirlo
    csv_out_file.close()
    #return idx
    idx = keypoints_per_part_of_body(values)
    return idx

#Classify the images
def classify_from_data():
    df = pd.read_csv(csv_out_path)
    y = df["class"]
    x = df.loc[:,df.columns!="class"]
    x = x.loc[:,x.columns!="name"]
    ###Splitting train/test data
    #display(x)
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=25/100,random_state=109)
    ###Creating Support Vector Machine Model
    clf = svm.SVC(kernel='rbf')
    ###Training the Model
    clf.fit(X_tr, y_tr)
    ###Making Predictions
    y_pr = clf.predict(X_tst)
    return clf, y_tst, y_pr

#Try the interface in real time
def loop(clf, idx):            
    global stop_flag
    # Initialize the webcam
    camera = cv2.VideoCapture(0)
    # Check if the webcam is opened correctly
    if not camera.isOpened():
        print("Cannot open camera")
        window['-3-'].update("Cannot open camera")
        exit()
    stopped = False
    while not stopped:
        event, values = window.read(timeout=0)  # Add timeout=0 to avoid GUI freezing
        if event == 'STOP':
            stop_flag = True
            window['-3-'].update("Loop stopped.")
        pose_landmarks_csv = []
        pose_landmarks_img = []
        # Read a frame from the webcam
        ret, image = camera.read()
        image_cam = image
        success, encoded_image = cv2.imencode('.jpg', image)
        if not success:
            print('Error encoding image')
            continue
        if stop_flag:
            stopped = True
        image = tf.io.decode_image(encoded_image.tobytes(), channels=3, expand_animations=False)
        image = tf.expand_dims(image, axis=0)
        # Resize and pad the image to keep the aspect ratio and fit the expected size.
        image = tf.cast(tf.image.resize_with_pad(image, 192, 192), dtype=tf.int32)
        # Run model inference.
        outputs = movenet(image)
        idx_tensor = tf.constant(idx, dtype=tf.int32)
        keypoints = tf.gather(outputs['output_0'], idx_tensor, axis=2)
        #break       
        if keypoints is not None:
            pose_np = np.squeeze(keypoints.numpy())  # Remove the batch and channel dimensions
            pose_np = pose_np.transpose()
            for i in range(pose_np.shape[1]):
                pose_landmarks_csv.append(pose_np[0, i])
                pose_landmarks_csv.append(pose_np[1, i])
            for i in range(pose_np.shape[0]):
                pose_landmarks_img.append((int(pose_np[i][0]), int(pose_np[i][1])))
        # Classify pose
        y_pr = clf.predict(pd.DataFrame(pose_landmarks_csv).transpose())
        i = y_pr[0]
        if i == 'clase0' and flag0 == 0:
            #display(audio1)
            #pay_audio(path_audio1)
            #clear_output(wait = True)
            start = time.time()
            audio_thread = threading.Thread(target=play_audio, args=(1,))
            audio_thread.start()
            window['-3-'].update("Clase 0.")
            flag0 = 1
            flag1 = 0
        elif i == 'clase1' and flag1 == 0:
            start = time.time()
            #display(audio6)
            play_audio(path_audio6)
            #clear_output(wait = True)
            start = time.time()
            audio_thread = threading.Thread(target=play_audio, args=(2,))
            audio_thread.start()
            print(time.time() - start)
            window['-3-'].update("Clase 1.")
            flag1 = 1
            flag0 = 0
    camera.release()
    window['-3-'].update("Loop stopped.")

In [9]:
## PRIMERA VENTANA

# Event loop to process GUI events
window = create_classes_window()
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED:
        break
    if event == 'Create Classes':
        window['-OUTPUT-'].update('Creating classes...')
        window.refresh()
        # Initialize the webcam
        camera = cv2.VideoCapture(0)
        # Close if we try to crate new data without specifying the size
        try:
            num_classes = int(values[0])
            num_photos = int(values[1])
        except ValueError:
            window.close()
            break
        # Define the number of photos to take and the time between photos
        num_classes = int(values[0])
        num_photos = int(values[1])
        time_between_photos = 0.5
        create_classes(window, path_to_data, path_to_classes, num_classes, num_photos, time_between_photos)       
        # Release the webcam
        camera.release()
        window['-OUTPUT-'].update('Classes created')
        time.sleep(2)
        window.close()
    elif event == 'Close window':
        break        
# Close the window when the event loop is exited
window.close()
#SEGUNDA VENTANA
# Crear la ventana y mostrarla
window = create_training_window()
stop_flag = True
while True:
    event, values = window.read()
    if event == sg.WINDOW_CLOSED:
        break
    elif event == 'Train Model':
        with open(csv_out_path, 'w') as csv_out_file:
            idx = extract_features_to_train(csv_out_file, images_folder, images_out_folder)
            #print('idx', idx)
            window['-1-'].update('Model trained')
    elif event == 'Classify Pose from Camera':
        ###Evaluating Prediction Accuracy
        clf, y_tst, y_pr = classify_from_data()
        window['-2-'].update(f'Pose classified from camera. Accuracy: {metrics.accuracy_score(y_tst, y_pr)}')
        print('Pose classified from camera. Accuracy:', metrics.accuracy_score(y_tst, y_pr))
window.Close()

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 16.15it/s]


,name,class,x1,y1,x2,y2,x3,y3,x4,y4,...,x13,y13,x14,y14,x15,y15,x16,y16,x17,y17
0,photo_0.png,clase0,0.368673,0.502184,0.355960,0.519382,0.355281,0.485392,0.374582,0.538968,...,0.758424,0.429177,0.889983,0.573008,0.876509,0.400985,0.835944,0.591638,0.837634,0.389894
1,photo_1.png,clase0,0.364871,0.497986,0.353217,0.512170,0.353346,0.482495,0.370270,0.535771,...,0.757186,0.427826,0.887455,0.572092,0.878092,0.401754,0.831797,0.603327,0.840084,0.388534
2,photo_2.png,clase0,0.370135,0.500990,0.356344,0.517753,0.356209,0.484852,0.373570,0.538435,...,0.758510,0.429033,0.886501,0.566186,0.875018,0.398602,0.816406,0.627943,0.836391,0.388804
3,photo_3.png,clase0,0.365027,0.494630,0.353965,0.509318,0.353007,0.477024,0.370606,0.535248,...,0.762850,0.427689,0.889922,0.569716,0.877645,0.396214,0.834516,0.582788,0.839137,0.380030
4,photo_4.png,clase0,0.365776,0.496652,0.353743,0.510081,0.353879,0.478434,0.370134,0.535256,...,0.755021,0.428184,0.885839,0.571586,0.876119,0.403599,0.836106,0.588066,0.837861,0.389417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,photo_3.png,clase7,0.399485,0.529535,0.379771,0.545341,0.381507,0.511354,0.382111,0.573547,...,0.753590,0.478050,0.881629,0.612357,0.876674,0.438293,0.851019,0.594429,0.837294,0.476010
60,photo_4.png,clase7,0.400822,0.528725,0.380550,0.543886,0.383227,0.509729,0.381868,0.571124,...,0.752177,0.477526,0.877959,0.613394,0.874728,0.436972,0.845651,0.611011,0.833660,0.472809
61,photo_5.png,clase7,0.400830,0.528966,0.380664,0.544059,0.383441,0.510083,0.381498,0.571932,...,0.750023,0.476965,0.877610,0.612417,0.874945,0.436385,0.837796,0.618646,0.835285,0.475033
62,photo_6.png,clase7,0.401533,0.529217,0.381456,0.544752,0.383607,0.510299,0.381079,0.572467,...,0.751382,0.474416,0.878837,0.611454,0.876690,0.435173,0.837300,0.617992,0.839328,0.454815


,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,x13,y13,x14,y14,x15,y15,x16,y16,x17,y17
0,0.368673,0.502184,0.355960,0.519382,0.355281,0.485392,0.374582,0.538968,0.373780,0.464585,...,0.758424,0.429177,0.889983,0.573008,0.876509,0.400985,0.835944,0.591638,0.837634,0.389894
1,0.364871,0.497986,0.353217,0.512170,0.353346,0.482495,0.370270,0.535771,0.373975,0.460000,...,0.757186,0.427826,0.887455,0.572092,0.878092,0.401754,0.831797,0.603327,0.840084,0.388534
2,0.370135,0.500990,0.356344,0.517753,0.356209,0.484852,0.373570,0.538435,0.373784,0.464329,...,0.758510,0.429033,0.886501,0.566186,0.875018,0.398602,0.816406,0.627943,0.836391,0.388804
3,0.365027,0.494630,0.353965,0.509318,0.353007,0.477024,0.370606,0.535248,0.372678,0.457199,...,0.762850,0.427689,0.889922,0.569716,0.877645,0.396214,0.834516,0.582788,0.839137,0.380030
4,0.365776,0.496652,0.353743,0.510081,0.353879,0.478434,0.370134,0.535256,0.373736,0.457344,...,0.755021,0.428184,0.885839,0.571586,0.876119,0.403599,0.836106,0.588066,0.837861,0.389417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.399485,0.529535,0.379771,0.545341,0.381507,0.511354,0.382111,0.573547,0.392272,0.495719,...,0.753590,0.478050,0.881629,0.612357,0.876674,0.438293,0.851019,0.594429,0.837294,0.476010
60,0.400822,0.528725,0.380550,0.543886,0.383227,0.509729,0.381868,0.571124,0.394362,0.493681,...,0.752177,0.477526,0.877959,0.613394,0.874728,0.436972,0.845651,0.611011,0.833660,0.472809
61,0.400830,0.528966,0.380664,0.544059,0.383441,0.510083,0.381498,0.571932,0.394277,0.494504,...,0.750023,0.476965,0.877610,0.612417,0.874945,0.436385,0.837796,0.618646,0.835285,0.475033
62,0.401533,0.529217,0.381456,0.544752,0.383607,0.510299,0.381079,0.572467,0.392144,0.494143,...,0.751382,0.474416,0.878837,0.611454,0.876690,0.435173,0.837300,0.617992,0.839328,0.454815


Pose classified from camera. Accuracy: 0.125


**Create test images**

In [9]:
# Path to the folder containing the input images
#images_folder_test = directory +'Rehabilitation_interface\\HandsEvaluation\\test\\Data\\' #1
# Path to the folder containing the body keypoints
#images_out_folder_test = directory +'Rehabilitation_interface\\HandsEvaluation\\test\\DataOutputMovenet\\' #2
# Path to the output CSV file containing the body keypoints
csv_out_path_test = directory +'Rehabilitation_interface\\BodyEvaluation\\test\\fitness_poses_csvs_out_basic_movenet.csv' #4
# Path to the images
path_to_data_test = directory +'Rehabilitation_interface\\BodyEvaluation\\test\\Data'
path_to_classes_test = directory +'Rehabilitation_interface\\BodyEvaluation\\test\\Data'
# Path to the folder containing the input images
images_folder_test = directory +'Rehabilitation_interface\\BodyEvaluation\\test\\' #1
images_out_folder_test = directory +'Rehabilitation_interface\\BodyEvaluation\\test\\' #2

# Event loop to process GUI events
window = create_classes_window()
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED:
        break
    if event == 'Create Classes':
        window['-OUTPUT-'].update('Creating classes...')
        window.refresh()
        # Initialize the webcam
        camera = cv2.VideoCapture(0)
        # Close if we try to crate new data without specifying the size
        try:
            num_classes = int(values[0])
            num_photos = int(values[1])
        except ValueError:
            window.close()
            break
        # Define the number of photos to take and the time between photos
        num_classes = int(values[0])
        num_photos = int(values[1])
        time_between_photos = 0.5
        create_classes(window, path_to_data_test, path_to_classes_test, num_classes, num_photos, time_between_photos)       
        # Release the webcam
        camera.release()
        window['-OUTPUT-'].update('Classes created')
        time.sleep(2)
        window.close()
    elif event == 'Close window':
        break        

In [11]:
def extract_features_to_test(csv_out_file, images_folder, images_out_foulder):
    csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    # Folder names are used as pose class names.
    pose_class_names = sorted([n for n in os.listdir(images_folder) if not n.startswith('.')])
    # Write the header row to the CSV file
    #csv_out_writer.writerow(['name', 'class', 'x1', 'y1', 'p1', 'x2', 'y2', 'p2', 'x3', 'y3', 'p3', 'x4', 'y4', 'p4', 'x5', 'y5', 'p5', 'x6', 'y6', 'p6', 'x7', 'y7', 'p7', 'x8', 'y8', 'p8', 'x9', 'y9', 'p9', 'x10', 'y10', 'p10', 'x11', 'y11', 'p11', 'x12', 'y12', 'p12', 'x13', 'y13', 'p13', 'x14', 'y14', 'p14', 'x15', 'y15', 'p15', 'x16', 'y16', 'p16', 'x17', 'y17', 'p17'])  # Header row
    csv_out_writer.writerow(['name', 'class', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7', 'x8', 'y8', 'x9', 'y9', 'x10', 'y10', 'x11', 'y11', 'x12', 'y12', 'x13', 'y13', 'x14', 'y14', 'x15', 'y15', 'x16', 'y16', 'x17', 'y17'])  # Header row
    #for pose_class_name in pose_class_names[1:]:
    message = f'Bootstrapping test '
    #window['-1-'].update(message)
    #if not os.path.exists(os.path.join(images_out_folder, pose_class_name)):
        #os.makedirs(os.path.join(images_out_folder, pose_class_name))
    image_names = sorted([
    n for n in os.listdir(os.path.join(images_folder))#, pose_class_name))
    if not n.startswith('.')])
    new_row = []
    pose_class_name = ''
    #print(image_names)
    for image_name in tqdm.tqdm(image_names[1:], position=0):      
        new_row = []
        # Extract the landmarks from the image
        landmarks = extract_landmarks(os.path.join(images_folder,pose_class_name, image_name))
        # Assuming you have the pose data in a variable named `pose`
        pose_np = np.squeeze(landmarks.numpy())  # Remove the batch and channel dimensions
        pose_np = pose_np.transpose()
        # Write pose sample to CSV.
        new_row.append(image_name)
        new_row.append('clase'+image_name[6:7])
        for i in range(pose_np.shape[1]):
            x = pose_np[0,i]
            y = pose_np[1,i]
            #p = pose_np[2,i]
            new_row.append(x)
            new_row.append(y)
            #new_row.append(p)
        # Save the landmarks to the CSV file
        csv_out_writer.writerow(new_row)
    # Aqui quiero cambiar todo el fichero csv_out_path y sobreescribirlo
    csv_out_file.close()
    #return idx
    idx = keypoints_per_part_of_body(values)
    return idx
with open(csv_out_path_test, 'w') as csv_out_file_test:
    idx_T = extract_features_to_test(csv_out_file_test, images_folder_test, images_out_folder_test)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 39.16it/s]


In [12]:
pd.read_csv(csv_out_path_test).head()

,name,class,x1,y1,x2,y2,x3,y3,x4,y4,...,x13,y13,x14,y14,x15,y15,x16,y16,x17,y17
0,photo_0.jpg,clase0,0.301093,0.461867,0.286429,0.482126,0.285575,0.443656,0.306233,0.498824,...,0.704964,0.397958,0.784000,0.572658,0.798092,0.377345,0.758444,0.589536,0.744623,0.325061
1,photo_1.jpg,clase1,0.293399,0.487137,0.277063,0.503699,0.276823,0.467843,0.296969,0.524246,...,0.681367,0.441763,0.770046,0.581619,0.784000,0.434753,0.733458,0.622006,0.731500,0.606595
2,photo_2.jpg,clase2,0.319678,0.536045,0.308382,0.555594,0.306898,0.521864,0.328702,0.577818,...,0.703794,0.470084,0.787738,0.641640,0.793832,0.456316,0.745095,0.657493,0.741919,0.635755
3,photo_3.jpg,clase3,0.318403,0.506821,0.308034,0.528300,0.305195,0.491157,0.326069,0.549220,...,0.703301,0.432282,0.783064,0.602043,0.784241,0.440556,0.762234,0.628877,0.757910,0.418388
4,photo_4.jpg,clase4,0.358218,0.582890,0.342157,0.600577,0.342347,0.568711,0.354181,0.623216,...,0.714518,0.505036,0.781695,0.660406,0.783979,0.485958,0.749152,0.692934,0.765085,0.509495


## Different algorithms accuracy

### SVM

In [14]:
import pandas as pd
from sklearn import metrics
from itertools import combinations
#SVM
def classify_from_data_df_svm(df):
    y = df["class"]
    x = df.loc[:,df.columns!="class"]
    x = x.loc[:,x.columns!="name"]
    ###Splitting train/test data
    #display(x)
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=25/100,random_state=109)
    ###Creating Support Vector Machine Model
    clf = svm.SVC(kernel='rbf')
    ###Training the Model
    clf.fit(X_tr, y_tr)
    ###Making Predictions
    y_pr = clf.predict(X_tst)
    return clf, y_tst, y_pr

# Leer el dataframe
df = pd.read_csv(csv_out_path)

# Obtener todas las clases únicas
classes = df['class'].unique()

acc = []
accTemp = []
# Realizar la evaluación para todas las combinaciones posibles
for r in range(2, len(classes) + 1):
    combinations_r = combinations(classes, r)
    for combination in combinations_r:
        # Filtrar el dataframe para las clases seleccionadas
        df2 = df[df['class'].isin(combination)]
        
        # Realizar la clasificación y calcular la precisión
        clf, y_tst, y_pr = classify_from_data_df_svm(df2)
        accuracy = metrics.accuracy_score(y_tst, y_pr)
        
        # Imprimir los resultados
        class_str = ', '.join(combination)
        #print(f'Pose classified from camera. Clases: {class_str}. Accuracy: {accuracy}')
        accTemp.append(accuracy)
        acc.append(accuracy)
    print("BODY SVM: Mean accuracy with all the combinations of ", r, " classes: ", np.mean(accTemp))
    accTemp = []
        
print("--------------------")
print("BODY SVM: Mean accuracy of ALL the combinations of classes: ", np.mean(acc))
df_test = pd.read_csv('C:\\Users\\rache\\Documents\\MusicTFM\\svmAlgorithm\\test\\fitness_poses_csvs_out_basic_movenet.csv')
y_test = df_test["class"]
x_test = df_test.loc[:, df_test.columns != "class"]
x_test = x_test.loc[:, x_test.columns != "name"]
# Making test predictions
y = df["class"]
x = df.loc[:, df.columns != "class"]
x = x.loc[:, x.columns != "name"]

# Splitting train/test data
X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)

# Creating AdaBoost model
clf = svm.SVC(kernel='rbf')

# Training the model
clf.fit(X_tr, y_tr)
y_test_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("--------------------")
print('y_test', y_test)
print('y_test_pred', y_test_pred)
print("--------------------")
print('Accuracy on new test data: ', accuracy)

BODY SVM: Mean accuracy with all the combinations of  2  classes:  0.6785714285714286
BODY SVM: Mean accuracy with all the combinations of  3  classes:  0.5297619047619048
BODY SVM: Mean accuracy with all the combinations of  4  classes:  0.4660714285714286
BODY SVM: Mean accuracy with all the combinations of  5  classes:  0.3392857142857143
BODY SVM: Mean accuracy with all the combinations of  6  classes:  0.2857142857142857
BODY SVM: Mean accuracy with all the combinations of  7  classes:  0.1875
BODY SVM: Mean accuracy with all the combinations of  8  classes:  0.125
--------------------
BODY SVM: Mean accuracy of ALL the combinations of classes:  0.44500674763832665
--------------------
y_test 0    clase0
1    clase1
2    clase2
3    clase3
4    clase4
5    clase5
6    clase6
7    clase7
Name: class, dtype: object
y_test_pred ['clase0' 'clase0' 'clase0' 'clase2' 'clase6' 'clase6' 'clase6' 'clase6']
--------------------
Accuracy on new test data:  0.25


### KNN

In [15]:
from sklearn.neighbors import KNeighborsClassifier

def classify_from_data_df_knn(df):
    y = df["class"]
    x = df.loc[:, df.columns != "class"]
    x = x.loc[:, x.columns != "name"]
    
    # Splitting train/test data
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)
    
    # Creating KNN model
    clf = KNeighborsClassifier(n_neighbors=5)
    
    # Training the model
    clf.fit(X_tr, y_tr)
    
    # Making predictions
    y_pr = clf.predict(X_tst)
    
    return clf, y_tst, y_pr

# Leer el dataframe
df = pd.read_csv(csv_out_path)

# Obtener todas las clases únicas
classes = df['class'].unique()

acc = []
accTemp = []
# Realizar la evaluación para todas las combinaciones posibles
for r in range(2, len(classes) + 1):
    combinations_r = combinations(classes, r)
    for combination in combinations_r:
        # Filtrar el dataframe para las clases seleccionadas
        df2 = df[df['class'].isin(combination)]
        
        # Realizar la clasificación y calcular la precisión
        clf, y_tst, y_pr = classify_from_data_df_knn(df2)
        accuracy = metrics.accuracy_score(y_tst, y_pr)
        
        # Imprimir los resultados
        class_str = ', '.join(combination)
        #print(f'Pose classified from camera. Clases: {class_str}. Accuracy: {accuracy}')
        accTemp.append(accuracy)
        acc.append(accuracy)
    print("BODY KNN: Mean accuracy with all the combinations of ", r, " classes: ", np.mean(accTemp))
    accTemp = []
        
print("--------------------")
print("BODY KNN: Mean accuracy of ALL the combinations of classes: ", np.mean(acc))
df_test = pd.read_csv('C:\\Users\\rache\\Documents\\MusicTFM\\svmAlgorithm\\test\\fitness_poses_csvs_out_basic_movenet.csv')
y_test = df_test["class"]
x_test = df_test.loc[:, df_test.columns != "class"]
x_test = x_test.loc[:, x_test.columns != "name"]
# Making test predictions
y = df["class"]
x = df.loc[:, df.columns != "class"]
x = x.loc[:, x.columns != "name"]

# Splitting train/test data
X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)

# Creating AdaBoost model
clf = KNeighborsClassifier()

# Training the model
clf.fit(X_tr, y_tr)
y_test_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("--------------------")
print('y_test', y_test)
print('y_test_pred', y_test_pred)
print("--------------------")
print('Accuracy on new test data: ', accuracy)

BODY KNN: Mean accuracy with all the combinations of  2  classes:  0.9821428571428571
BODY KNN: Mean accuracy with all the combinations of  3  classes:  0.9791666666666667
BODY KNN: Mean accuracy with all the combinations of  4  classes:  0.9696428571428571
BODY KNN: Mean accuracy with all the combinations of  5  classes:  1.0
BODY KNN: Mean accuracy with all the combinations of  6  classes:  0.9583333333333334
BODY KNN: Mean accuracy with all the combinations of  7  classes:  0.9464285714285714
BODY KNN: Mean accuracy with all the combinations of  8  classes:  1.0
--------------------
BODY KNN: Mean accuracy of ALL the combinations of classes:  0.9781906689801427
--------------------
y_test 0    clase0
1    clase1
2    clase2
3    clase3
4    clase4
5    clase5
6    clase6
7    clase7
Name: class, dtype: object
y_test_pred ['clase0' 'clase2' 'clase2' 'clase3' 'clase4' 'clase6' 'clase6' 'clase7']
--------------------
Accuracy on new test data:  0.75


### Decision tree

In [16]:

from sklearn.tree import DecisionTreeClassifier

def classify_from_data_df_decision_tree(df):
    y = df["class"]
    x = df.loc[:, df.columns != "class"]
    x = x.loc[:, x.columns != "name"]
    
    # Splitting train/test data
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)
    
    # Creating Decision Tree model
    clf = DecisionTreeClassifier()
    
    # Training the model
    clf.fit(X_tr, y_tr)
    
    # Making predictions
    y_pr = clf.predict(X_tst)
    
    return clf, y_tst, y_pr

# Leer el dataframe
df = pd.read_csv(csv_out_path)

# Obtener todas las clases únicas
classes = df['class'].unique()

acc = []
accTemp = []
# Realizar la evaluación para todas las combinaciones posibles
for r in range(2, len(classes) + 1):
    combinations_r = combinations(classes, r)
    for combination in combinations_r:
        # Filtrar el dataframe para las clases seleccionadas
        df2 = df[df['class'].isin(combination)]
        
        # Realizar la clasificación y calcular la precisión
        clf, y_tst, y_pr = classify_from_data_df_decision_tree(df2)
        accuracy = metrics.accuracy_score(y_tst, y_pr)
        
        # Imprimir los resultados
        class_str = ', '.join(combination)
        #print(f'Pose classified from camera. Clases: {class_str}. Accuracy: {accuracy}')
        accTemp.append(accuracy)
        acc.append(accuracy)
    print("BODY DT: Mean accuracy with all the combinations of ", r, " classes: ", np.mean(accTemp))
    accTemp = []
                
print("--------------------")
print("BODY DT: Mean accuracy of ALL the combinations of classes: ", np.mean(acc))
df_test = pd.read_csv('C:\\Users\\rache\\Documents\\MusicTFM\\svmAlgorithm\\test\\fitness_poses_csvs_out_basic_movenet.csv')
y_test = df_test["class"]
x_test = df_test.loc[:, df_test.columns != "class"]
x_test = x_test.loc[:, x_test.columns != "name"]
# Making test predictions
y = df["class"]
x = df.loc[:, df.columns != "class"]
x = x.loc[:, x.columns != "name"]

# Splitting train/test data
X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)

# Creating AdaBoost model
clf = DecisionTreeClassifier()

# Training the model
clf.fit(X_tr, y_tr)
y_test_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("--------------------")
print('y_test', y_test)
print('y_test_pred', y_test_pred)
print("--------------------")
print('Accuracy on new test data: ', accuracy)

BODY DT: Mean accuracy with all the combinations of  2  classes:  0.9196428571428571
BODY DT: Mean accuracy with all the combinations of  3  classes:  0.9464285714285714
BODY DT: Mean accuracy with all the combinations of  4  classes:  0.8089285714285714
BODY DT: Mean accuracy with all the combinations of  5  classes:  0.9214285714285715
BODY DT: Mean accuracy with all the combinations of  6  classes:  0.8630952380952381
BODY DT: Mean accuracy with all the combinations of  7  classes:  0.8125
BODY DT: Mean accuracy with all the combinations of  8  classes:  0.625
--------------------
BODY DT: Mean accuracy of ALL the combinations of classes:  0.8836707152496626
--------------------
y_test 0    clase0
1    clase1
2    clase2
3    clase3
4    clase4
5    clase5
6    clase6
7    clase7
Name: class, dtype: object
y_test_pred ['clase0' 'clase0' 'clase2' 'clase0' 'clase6' 'clase6' 'clase7' 'clase7']
--------------------
Accuracy on new test data:  0.375


### Ada Boost

In [17]:
from sklearn.ensemble import AdaBoostClassifier

def classify_from_data_df_adaboost(df):
    y = df["class"]
    x = df.loc[:, df.columns != "class"]
    x = x.loc[:, x.columns != "name"]
    
    # Splitting train/test data
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)
    
    # Creating AdaBoost model
    clf = AdaBoostClassifier()
    
    # Training the model
    clf.fit(X_tr, y_tr)
    
    # Making predictions
    y_pr = clf.predict(X_tst)
    
    return clf, y_tst, y_pr
# Leer el dataframe
df = pd.read_csv(csv_out_path)

# Obtener todas las clases únicas
classes = df['class'].unique()

acc = []
accTemp = []
# Realizar la evaluación para todas las combinaciones posibles
for r in range(2, len(classes) + 1):
    combinations_r = combinations(classes, r)
    for combination in combinations_r:
        # Filtrar el dataframe para las clases seleccionadas
        df2 = df[df['class'].isin(combination)]
        
        # Realizar la clasificación y calcular la precisión
        clf, y_tst, y_pr = classify_from_data_df_adaboost(df2)
        accuracy = metrics.accuracy_score(y_tst, y_pr)
        
        # Imprimir los resultados
        class_str = ', '.join(combination)
        #print(f'Pose classified from camera. Clases: {class_str}. Accuracy: {accuracy}')
        accTemp.append(accuracy)
        acc.append(accuracy)
    print("BODY AB: Mean accuracy with all the combinations of ", r, " classes: ", np.mean(accTemp))
    accTemp = []
        
print("--------------------")
print("BODY AB: Mean accuracy of ALL the combinations of classes: ", np.mean(acc))
df_test = pd.read_csv('C:\\Users\\rache\\Documents\\MusicTFM\\svmAlgorithm\\test\\fitness_poses_csvs_out_basic_movenet.csv')
y_test = df_test["class"]
x_test = df_test.loc[:, df_test.columns != "class"]
x_test = x_test.loc[:, x_test.columns != "name"]
# Making test predictions
y = df["class"]
x = df.loc[:, df.columns != "class"]
x = x.loc[:, x.columns != "name"]

# Splitting train/test data
X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)

# Creating AdaBoost model
clf = AdaBoostClassifier()

# Training the model
clf.fit(X_tr, y_tr)
y_test_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("--------------------")
print('y_test', y_test)
print('y_test_pred', y_test_pred)
print("--------------------")
print('Accuracy on new test data: ', accuracy)

BODY AB: Mean accuracy with all the combinations of  2  classes:  0.9107142857142857
BODY AB: Mean accuracy with all the combinations of  3  classes:  0.988095238095238
BODY AB: Mean accuracy with all the combinations of  4  classes:  0.9410714285714286
BODY AB: Mean accuracy with all the combinations of  5  classes:  0.8857142857142856
BODY AB: Mean accuracy with all the combinations of  6  classes:  0.6279761904761906
BODY AB: Mean accuracy with all the combinations of  7  classes:  0.5
BODY AB: Mean accuracy with all the combinations of  8  classes:  0.3125
--------------------
BODY AB: Mean accuracy of ALL the combinations of classes:  0.8834176788124158
--------------------
y_test 0    clase0
1    clase1
2    clase2
3    clase3
4    clase4
5    clase5
6    clase6
7    clase7
Name: class, dtype: object
y_test_pred ['clase4' 'clase2' 'clase2' 'clase2' 'clase6' 'clase6' 'clase2' 'clase7']
--------------------
Accuracy on new test data:  0.25


### Naive Bayes

In [18]:
#naive bayes
from sklearn.naive_bayes import GaussianNB

def classify_from_data_df_naive_bayes(df):
    y = df["class"]
    x = df.loc[:, df.columns != "class"]
    x = x.loc[:, x.columns != "name"]
    
    # Splitting train/test data
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)
    
    # Creating Naive Bayes model
    clf = GaussianNB()
    
    # Training the model
    clf.fit(X_tr, y_tr)
    
    # Making predictions
    y_pr = clf.predict(X_tst)
    
    return clf, y_tst, y_pr
# Leer el dataframe
df = pd.read_csv(csv_out_path)

# Obtener todas las clases únicas
classes = df['class'].unique()

acc = []
accTemp = []
# Realizar la evaluación para todas las combinaciones posibles
for r in range(2, len(classes) + 1):
    combinations_r = combinations(classes, r)
    for combination in combinations_r:
        # Filtrar el dataframe para las clases seleccionadas
        df2 = df[df['class'].isin(combination)]
        
        # Realizar la clasificación y calcular la precisión
        clf, y_tst, y_pr = classify_from_data_df_naive_bayes(df2)
        accuracy = metrics.accuracy_score(y_tst, y_pr)
        
        # Imprimir los resultados
        class_str = ', '.join(combination)
        #print(f'Pose classified from camera. Clases: {class_str}. Accuracy: {accuracy}')
        accTemp.append(accuracy)
        acc.append(accuracy)
    print("BODY NB: Mean accuracy with all the combinations of ", r, " classes: ", np.mean(accTemp))
    accTemp = []
print("--------------------")
print("BODY NB: Mean accuracy of ALL the combinations of classes: ", np.mean(acc))
df_test = pd.read_csv('C:\\Users\\rache\\Documents\\MusicTFM\\svmAlgorithm\\test\\fitness_poses_csvs_out_basic_movenet.csv')
y_test = df_test["class"]
x_test = df_test.loc[:, df_test.columns != "class"]
x_test = x_test.loc[:, x_test.columns != "name"]
# Making test predictions
y = df["class"]
x = df.loc[:, df.columns != "class"]
x = x.loc[:, x.columns != "name"]

# Splitting train/test data
X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)

# Creating AdaBoost model
clf = GaussianNB()

# Training the model
clf.fit(X_tr, y_tr)
y_test_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("--------------------")
print('y_test', y_test)
print('y_test_pred', y_test_pred)
print("--------------------")
print('Accuracy on new test data: ', accuracy)

BODY NB: Mean accuracy with all the combinations of  2  classes:  0.9553571428571429
BODY NB: Mean accuracy with all the combinations of  3  classes:  0.9345238095238094
BODY NB: Mean accuracy with all the combinations of  4  classes:  0.9625
BODY NB: Mean accuracy with all the combinations of  5  classes:  1.0
BODY NB: Mean accuracy with all the combinations of  6  classes:  0.9374999999999999
BODY NB: Mean accuracy with all the combinations of  7  classes:  1.0
BODY NB: Mean accuracy with all the combinations of  8  classes:  0.9375
--------------------
BODY NB: Mean accuracy of ALL the combinations of classes:  0.9621288798920379
--------------------
y_test 0    clase0
1    clase1
2    clase2
3    clase3
4    clase4
5    clase5
6    clase6
7    clase7
Name: class, dtype: object
y_test_pred ['clase1' 'clase2' 'clase2' 'clase6' 'clase6' 'clase6' 'clase6' 'clase6']
--------------------
Accuracy on new test data:  0.25
